<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/%EB%B9%85%EC%9D%91%EB%B3%B4_case4_AE_IF_LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:

data = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type4.csv')#.to_numpy()


X = data[data['type'] == 'Normal']
y = X['type']
y = y.map({'Normal': 0, 'Attack': 1}).to_numpy()
X = X.drop(columns =['type'])
X['messageID'] = X['messageID'].astype('category')
X = X.drop(columns =['isCan0','isService','serviceNODEID'])
X = X.drop(columns = ['data[0]','data[7]'])
print(X.shape)


(104204, 12)


In [20]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]
encoding_dim = 7

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Autoencoder 학습
autoencoder.fit(X_scaled, X_scaled, epochs=30, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/30
2606/2606 [==============================] - 7s 3ms/step - loss: 0.6934 - val_loss: 0.7863
Epoch 2/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5912 - val_loss: 0.7030
Epoch 3/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5828 - val_loss: 0.6808
Epoch 4/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5794 - val_loss: 0.6764
Epoch 5/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5783 - val_loss: 0.6630
Epoch 6/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5776 - val_loss: 0.6592
Epoch 7/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5771 - val_loss: 0.6560
Epoch 8/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5756 - val_loss: 0.6694
Epoch 9/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.5750 - val_loss: 0.6771
Epoch 10/30
2606/2606 [==============================] - 6s 2ms/step - loss: 0.574

In [21]:
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
encoder_model = Model(inputs=input_layer, outputs=encoder)

# 데이터를 인코딩
X_encoded = autoencoder.predict(X_scaled)

3257/3257 [==============================] - 4s 1ms/step


In [22]:
#data.loc[data['type']=='Normal', 'type'] = 0
#data.loc[data['type']=='Attack', 'type'] = 1

fullDataX1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type4.csv')#.to_numpy()

#X['messageID'] = data['messageID']
fX1 = fullDataX1.drop(columns =['type'])
fX1 = fX1.drop(columns =['isCan0','isService','serviceNODEID'])
fX1 = fX1.drop(columns = ['data[0]','data[7]'])
print(fX1.shape)
fX1_scaled = scaler.fit_transform(fX1)
fX1_encoded = autoencoder.predict(fX1_scaled)

(133374, 12)
4168/4168 [==============================] - 5s 1ms/step


In [23]:
if_model = IsolationForest(contamination=0.1)  # contamination 값 조정 가능
if_model.fit(X_encoded)

# 이상치 예측
if_predictions = if_model.predict(fX1_encoded)
fy = fullDataX1['type']
fy = fy.map({'Normal': 1, 'Attack': 0}).to_numpy()
if_predictions = np.where(if_predictions == -1, 0, if_predictions)
print(np.unique(if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy, if_predictions)
class_report = classification_report(fy, if_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

(array([0, 1]), array([42266, 91108]))
Confusion Matrix:
[[24526  4644]
 [17740 86464]]

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.84      0.69     29170
           1       0.95      0.83      0.89    104204

    accuracy                           0.83    133374
   macro avg       0.76      0.84      0.79    133374
weighted avg       0.87      0.83      0.84    133374



In [24]:
# 모델 저장
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case4_if_model.pkl', 'wb') as file:
    #pickle.dump(if_model, file)

In [6]:
from sklearn.model_selection import GridSearchCV
if_model = IsolationForest(contamination=0.2)  # contamination 값 조정 가능

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': ['auto', 0.5, 0.75],
    'contamination': [0.05, 0.1, 0.2],
    'max_features': [1.0, 0.5, 0.75],
    'bootstrap': [False, True]
}

# GridSearchCV 사용
grid_search = GridSearchCV(estimator=if_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_scaled, y)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", grid_search.best_params_)

# 이상치 예측


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found:  {'bootstrap': False, 'contamination': 0.05, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 50}


In [7]:
best_params = grid_search.best_params_
best_if_model = IsolationForest(**best_params,)
best_if_model.fit(X_scaled)

#import joblib
#joblib.dump(best_if_model, './best_if_model.pkl')

# 모델 로드 및 novelty=True로 설정
if_model =best_if_model #joblib.load('./best_if_model.pkl')
#if_model.set_params(novelty=True)
if_predictions = if_model.predict(fX1_encoded)

In [8]:
from sklearn.metrics import confusion_matrix, classification_report
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 1, 'Attack': 0}).to_numpy()
if_predictions = np.where(if_predictions == -1, 0, if_predictions)
print(np.unique(if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, if_predictions)
class_report = classification_report(fy1, if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)
print(np.unique(if_predictions, return_counts=True))

(array([0, 1]), array([     4, 133370]))
Confusion Matrix:
[[     3  29167]
 [     1 104203]]

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.00      0.00     29170
           1       0.78      1.00      0.88    104204

    accuracy                           0.78    133374
   macro avg       0.77      0.50      0.44    133374
weighted avg       0.77      0.78      0.69    133374

(array([0, 1]), array([     4, 133370]))


In [9]:
# 모델 저장
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_if_model.pkl', 'wb') as file:
#    pickle.dump(if_model, file)

In [10]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case3_if_model.pkl', 'rb') as file:
    loaded_if_model = pickle.load(file)

# 불러온 모델로 예측
fy1 = fullDataX1['type']
fy1 = fy1.map({'Normal': 0, 'Attack': 1}).to_numpy()
loaded_if_predictions = loaded_if_model.predict(fX1_encoded)
loaded_if_predictions = np.where(loaded_if_predictions == -1, 0, loaded_if_predictions)
print(np.unique(loaded_if_predictions, return_counts=True))
conf_matrix = confusion_matrix(fy1, loaded_if_predictions)
class_report = classification_report(fy1, loaded_if_predictions)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

ValueError: X has 14 features, but IsolationForest is expecting 15 features as input.

In [ ]:
# Local Outlier Factor 모델 정의 및 학습
lof_model = LocalOutlierFactor(n_neighbors=30, contamination=0.1, novelty=True)  # n_neighbors 및 contamination 값 조정 가능
lof_model.fit(X_encoded)
lof_predictions = lof_model.predict(fX1_encoded)
# 이상치라면 -1, 정상치라면 1로 표시됨


In [ ]:
# 모델 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case4_lof_model.pkl', 'wb') as file:
    pickle.dump(lof_model, file)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/team_prj/case4_lof_model.pkl', 'rb') as file:
    loaded_lof_model = pickle.load(file)

# 불러온 모델로 예측
loaded_lof_predictions = loaded_lof_model.predict(fX1_encoded)
print(np.unique(loaded_lof_predictions, return_counts=True))

In [ ]:
print(np.unique(lof_predictions, return_counts=True))